In [1]:
import numpy as np
np.random.seed(43)
from keras.layers import Convolution2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.models import model_from_json
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.
/Users/ahmetcanozbek/anaconda/envs/qs/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Load the data
X = np.load('./X.npy')
y = np.load('./y.npy')

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

print 'X_train.shape:', X_train.shape
print 'y_train.shape:', y_train.shape
print 'X_test.shape:', X_test.shape
print 'y_test.shape:', y_test.shape

X_train.shape: (7315, 128, 128, 1)
y_train.shape: (7315,)
X_test.shape: (1829, 128, 128, 1)
y_test.shape: (1829,)


In [3]:
X_train = 1.0 - X_train
X_test = 1.0 - X_test

In [4]:
# Define the model
model = Sequential()
model.add(Convolution2D(6,5,5, border_mode='same', input_shape=(128,128,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(16,5,5, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(30,5,5, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(output_dim=5000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(output_dim=1000, activation='relu'))
model.add(Dense(output_dim=9, activation='softmax'))

# Define the optimizer and the loss function
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
y_train_tc = to_categorical(y_train)[:,1:]
hist = model.fit(X_train, y_train_tc, batch_size=64, nb_epoch=6)

Epoch 1/6
7315/7315 [==============================] - 157s - loss: 0.2984 - acc: 0.9049   
Epoch 2/6
7315/7315 [==============================] - 168s - loss: 0.0364 - acc: 0.9899   
Epoch 3/6
7315/7315 [==============================] - 162s - loss: 0.0192 - acc: 0.9936   
Epoch 4/6
7315/7315 [==============================] - 179s - loss: 0.0101 - acc: 0.9966   
Epoch 5/6
7315/7315 [==============================] - 175s - loss: 0.0125 - acc: 0.9958   
Epoch 6/6
7315/7315 [==============================] - 172s - loss: 0.0033 - acc: 0.9995   


In [5]:
# Save the model
model_json_str = model.to_json()
open('./model2_arch.txt','w').write(model_json_str)
model.save_weights('./model2_weights.h5')

In [6]:
# Load the model and test
model_arch_str = open('./model2_arch.txt', 'r').read()
model_l = model_from_json(model_arch_str)
model_l.load_weights('./model2_weights.h5')

np.mean(y_test == model_l.predict_classes(X_test) + 1)

1829/1829 [==============================] - 10s    


0.99070530344450525